In [ ]:
# Copyright 2023 Nils Knieling
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building AI-powered applications using LangChain and Google Vertex AI

[![Open in Colab](https://img.shields.io/badge/Open%20in%20Colab-%23F9AB00.svg?logo=googlecolab&logoColor=white)](https://colab.research.google.com/github/Cyclenerd/toolbox/blob/master/notebooks/LangChain_VertexAI.ipynb)
[![Open in Vertex AI Workbench](https://img.shields.io/badge/Open%20in%20Vertex%20AI%20Workbench-%234285F4.svg?logo=googlecloud&logoColor=white)](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/Cyclenerd/toolbox/master/notebooks/LangChain_VertexAI.ipynb)
[![View on GitHub](https://img.shields.io/badge/View%20on%20GitHub-181717.svg?logo=github&logoColor=white)](https://github.com/Cyclenerd/toolbox/blob/master/notebooks/LangChain_VertexAI.ipynb)


## Install required packages

>⚠️ You may receive a warning to "Restart Runtime" after the packages are installed. Don't worry, the subsequent cells will help you restart the runtime.

In [ ]:
#@markdown ### Install dependencies

!pip install langchain==0.0.244
!pip install -U google-cloud-aiplatform==1.28.1 "shapely < 2.0.0"
!pip install google-cloud-storage==2.8.0
!pip install unstructured==0.8.4
!pip install faiss-cpu==1.7.4
!pip install chromadb==0.4.3

print("☑️ Done")

In [ ]:
#@markdown ### Restart

# Automatically restart kernel after installs so that your environment
# can access the new packages.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Setup Google Cloud environment


In [ ]:
# @markdown ✏️ Replace the placeholder text below:

# Please fill in these values.
project_id = "test-nils-ai"  # @param {type:"string"}
region = "us-central1"  # @param {type:"string"}
bucket = "test-nils-data-man"  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert bucket, "⚠️ Please provide a Google Cloud storage bucket"

# Configure gcloud.
!gcloud config set project "{project_id}"
!gcloud config set storage/parallel_composite_upload_enabled "True"

print("☑️ Done")

In [ ]:
#@markdown ### (Colab only!) Authenticate your Google Cloud Account

# Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

In [ ]:
#@markdown ###  Check authenticated user
current_user = !gcloud auth list \
  --filter="status:ACTIVE" \
  --format="value(account)" \
  --quiet

current_user = current_user[0]
print(f"Current user: {current_user}")

In [ ]:
#@markdown ### Enable APIs

# Enable APIs
my_google_apis = [
    "storage.googleapis.com",
    "aiplatform.googleapis.com",
]

for api in my_google_apis :
  print(f"Enable API: {api}")
  !gcloud services enable "{api}" \
    --project="{project_id}" \
    --quiet

print("☑️ OK")

In [ ]:
#@markdown ### Create storage bucket for data

#@markdown > Only necessary if the bucket does not already exist!
!gcloud storage buckets create 'gs://{bucket}' \
  --location='{region}' \
  --uniform-bucket-level-access \
  --quiet

print("☑️ Done")
print(f"Open in console: https://console.cloud.google.com/storage/browser/{bucket}")

## LangChain & Vertex AI

In [ ]:
#@markdown #### Import and print versions

import sys
print(f"☑️ Python: {sys.version}")

# Langchain
import langchain

print(f"☑️ LangChain version: {langchain.__version__}")

# Vertex AI
# https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm
from google.cloud import aiplatform, aiplatform_v1beta1
from langchain.llms import VertexAI

aiplatform.init(
    project=project_id,
    location=region,
)

print(f"☑️ Vertex AI SDK version: {aiplatform.__version__}")

## Staging

### Directory Loader

In [ ]:
# Load documents from bucket.
# This code snippet may run for a few (>5) minutes.

# https://python.langchain.com/docs/integrations/document_loaders/google_cloud_storage_directory
from langchain.document_loaders import GCSDirectoryLoader

loader = GCSDirectoryLoader(project_name=f"{project_id}", bucket=f"{bucket}")
documents = loader.load()

print(f"☑️ You have {len(documents)} documents.")

### Text Splitter



In [ ]:
# Split long text descriptions into smaller chunks that can fit into
# the API request size limit, as expected by the LLM providers.

# https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)

chunked = text_splitter.split_documents(documents)

print("Preview:")
print(chunked[0].page_content, "\n")
print(chunked[1].page_content)

print("☑️ Done")

### Embeddings

In [ ]:
# Generate the vector embeddings for each chunk of text.
# This code snippet may run for a few (>28) minutes.

# Utils
import time
from typing import List
from pydantic import BaseModel

# https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm
from langchain.embeddings import VertexAIEmbeddings

embeddings_service = VertexAIEmbeddings()

# Facebook AI Similarity Search (FAISS), local Vector Store
# https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/
from langchain.vectorstores import FAISS
# TODO: Use Vertex AI Matching Engine as Vector Store.
#       https://cloud.google.com/vertex-ai/docs/matching-engine/overview
#       https://python.langchain.com/docs/integrations/vectorstores/matchingengine

# Alternative...
# Chroma, the local AI-native open-source embedding database
# https://python.langchain.com/docs/integrations/vectorstores/chroma
from langchain.vectorstores import Chroma


# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the Vertex AI API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]


# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

# Embed your texts
# FAISS...
#db = FAISS.from_documents(chunked, embeddings)
# or Chroma...
db = Chroma.from_documents(chunked, embeddings, persist_directory="./chroma_db")
# TODO: Load only from disk
# db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

print("\n☑️ Done")

In [ ]:
# Expose index to the retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})
retriever

In [ ]:
# Test
docs = retriever.get_relevant_documents(
    "Command to show uptime."
)

print("\n\n".join([x.page_content[:400] for x in docs[:2]]))

## Query

In [ ]:
# @markdown Enter search query in a simple English text.
user_query = "What is the default shell and the command to show the uptime?"  # @param {type:"string"}

# Quick input validations.
assert user_query, "⚠️ Please input a valid input search text"

# Create chain to answer questions
from langchain.chains import RetrievalQA

# Custom promt template
# https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/
from langchain.prompts import PromptTemplate

prompt_template = """I want you to act as a Linux expert.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Make sure the answer is correct and don't output false content.

{context}

Question: {question}
Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

result = qa({"query": user_query})

print("Result: ")
print(result["result"])

print("Sources: ")
print(result["source_documents"])
